In [2]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
import os 
import requests
import json
from typing import Dict
import boto3

# 1. DONE -  get one T-2 months days taxi data
# 2. DONE- get one  T-2 months days weather data
# 3. DONE - upload to s3 (raw_data /to_processed/waether_data and raw_data/ taxi_data )
# 4. DONE - create functions - organize the code
# 5. creating a trigger

def get_taxi_data(formatted_datetime : str) -> Dict :
    """
    Retrives taxi data for the given date.
    Paramaters:
        formatted_datetime (str): The date in "YYYY-MM-DD" format
    Returns:
        Dict: A dictionary containing the taxi_data as a JSON
    """
    taxi_url = f"https://data.cityofchicago.org/resource/wrvz-psew.json?$where=trip_start_timestamp >= '{formatted_datetime}T00:00:00' AND trip_start_timestamp <= '{formatted_datetime}T23:59:59'&$limit=30000"
    headers = {"X-App-Token" : os.environ.get("CHICAGO_API_TOKEN")}
    response_taxi = requests.get(taxi_url)
    taxi_data = response_taxi.json()
    
    return taxi_data
    
def get_weather_data(formatted_datetime : str) -> Dict :
    """
    Retrieve weather data for a specific date and time.

    Parameters:
        formatted_datetime (str): A string representing the date and time in a format accepted by the API.
    
    Returns:
        Dict: A dictionary containing weather data including temperature, wind speed, rain, and precipitation.
    """
    weather_url = "https://archive-api.open-meteo.com/v1/era5"
    params = {
        "latitude" : 41.85,
        "longitude" : -87.65,
        "start_date" : formatted_datetime,
        "end_date" : formatted_datetime,
        "hourly" : "temperature_2m,wind_speed_10m,rain,precipitation"
    }
    
    response_weather = requests.get(weather_url,params = params)
    weather_data = response_weather.json()
    
    return weather_data

def upload_to_s3(data: Dict , folder_name : str, file_name )-> None:
    """
    Upload data to an S3 bucket.

    Parameters:
        data (Dict): A dictionary containing the data to be uploaded.
        folder_name (str): The name of the folder where the file will be stored in the S3 bucket.
        file_name (str): The name of the file.

    Returns:
        None
    """
    client = boto3.client("s3")
    client.put_object(
        Bucket = "cubixdatachicagobm",
        Key = f"raw_data/to_processed/{folder_name}/{file_name}",
        Body = json.dumps(data)
    )

    
def lambda_handler(event, context):
    current_datetime = datetime.now() - relativedelta(months=35)

    formatted_datetime = current_datetime.strftime("%Y-%m-%d")
    
    taxi_data_api_call = get_taxi_data(formatted_datetime) 
    weather_data_api_call = get_weather_data(formatted_datetime)  
    
    taxi_filename = f"taxi_raw_{formatted_datetime}.json"
    weather_filename = f"taxi_raw_{formatted_datetime}.json"
    
    upload_to_s3(data = taxi_data_api_call, file_name = taxi_filename , folder_name = "taxi_data" )
    print("Taxi data uploaded")
    upload_to_s3(data = weather_data_api_call, file_name = weather_filename , folder_name = "weather_data" )
    print("Weather data uploaded")
    
    